In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
import string

/home/yugal/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
from pycaret.classification import *

In [3]:
df = pd.read_csv("train.csv")
df

,Review,Rating
0,Great game when i was a kid I grew up watching...,5
1,Nice game,1
2,Balo game but bholo laguchi khelibaku gote khu...,5
3,This is the worst computer cheating game I've ...,1
4,Great,5
...,...,...
20995,افضل لعبة و لكن اريد ان يكون شجرة الجواهر,5
20996,Very nice and enjoy!!!,4
20997,It has lots of different games depending on yo...,5
20998,Has several very frustrating bugs not present ...,3


In [4]:
# !pip install deep_translator

In [5]:
test = pd.read_csv("test.csv")
test

,Review
0,This used to be my favorite game and it is sti...
1,Every single time i put the online multi-playe...
2,Determined to make us watch ads! Wouldn't be a...
3,Super great running game
4,"Hello NetEase, I really loved the game it dese..."
...,...
6995,This is like playing real football
6996,Please fix aim accuracy
6997,"Fix your game ranking and chat, this game suck..."
6998,give coins 500000


In [6]:
# from deep_translator import GoogleTranslator

# #simple function to detect and translate text 
# def detect_and_translate(text,target_lang):
#     translate_text = GoogleTranslator(source='auto', target='en').translate(text)
#     return translate_text

In [7]:
# df['Review']= df['Review'].apply(lambda x: detect_and_translate(x,'en'))

In [8]:
# test['Review']= test['Review'].apply(lambda x: detect_and_translate(x,'en'))

In [9]:
from emot.emo_unicode import UNICODE_EMOJI 
from emot.emo_unicode import EMOTICONS_EMO 

In [10]:
def remove_flag(text):
    emoji_pattern = re.compile(      # Macau flag
        u'([\U0001F1E6-\U0001F1FF])'     # emoticons
        "+", flags=re.UNICODE)
    return emoji_pattern.sub('', text)

In [11]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [12]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [13]:
# from spellchecker import SpellChecker

# spell = SpellChecker()
# def correct_spellings(text):
#     corrected_text = []
#     misspelled_words = spell.unknown(text.split())
#     for word in text.split():
#         if word in misspelled_words:
#             corrected_text.append(spell.correction(word))
#         else:
#             corrected_text.append(word)
#     return " ".join(corrected_text)

In [14]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers, make emoji into words.'''
    
    text = remove_flag(text)
    text = remove_html(text)
    
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    text = text.lower()
    text = remove_urls(text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [15]:
data_clean = pd.DataFrame(df.Review.apply(round1))
data_clean

,Review
0,great game when i was a kid i grew up watching...
1,nice game
2,balo game but bholo laguchi khelibaku gote khu...
3,this is the worst computer cheating game ive e...
4,great
...,...
20995,افضل لعبة و لكن اريد ان يكون شجرة الجواهر
20996,very nice and enjoy
20997,it has lots of different games depending on yo...
20998,has several very frustrating bugs not present ...


In [16]:
# !pip install pyspellchecker

In [17]:
test_clean = pd.DataFrame(test.Review.apply(round1))
test_clean

,Review
0,this used to be my favorite game and it is sti...
1,every single time i put the online multiplayer...
2,determined to make us watch ads wouldnt be an ...
3,super great running game
4,hello netease i really loved the game it deser...
...,...
6995,this is like playing real football
6996,please fix aim accuracy
6997,fix your game ranking and chat this game sucks...
6998,give coins


In [18]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [19]:
data_clean = pd.DataFrame(data_clean.Review.apply(round2))
data_clean

,Review
0,great game when i was a kid i grew up watching...
1,nice game
2,balo game but bholo laguchi khelibaku gote khu...
3,this is the worst computer cheating game ive e...
4,great
...,...
20995,افضل لعبة و لكن اريد ان يكون شجرة الجواهر
20996,very nice and enjoy
20997,it has lots of different games depending on yo...
20998,has several very frustrating bugs not present ...


In [20]:
test_clean = pd.DataFrame(test_clean.Review.apply(round1))
test_clean

,Review
0,this used to be my favorite game and it is sti...
1,every single time i put the online multiplayer...
2,determined to make us watch ads wouldnt be an ...
3,super great running game
4,hello netease i really loved the game it deser...
...,...
6995,this is like playing real football
6996,please fix aim accuracy
6997,fix your game ranking and chat this game sucks...
6998,give coins


In [21]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [22]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

data_clean["Review"] = data_clean['Review'].apply(lambda text: remove_stopwords(text))
data_clean

,Review
0,great game kid grew watching pokémon buying po...
1,nice game
2,balo game bholo laguchi khelibaku gote khusi a...
3,worst computer cheating game ive ever played u...
4,great
...,...
20995,افضل لعبة و لكن اريد ان يكون شجرة الجواهر
20996,nice enjoy
20997,lots different games depending style also cust...
20998,several frustrating bugs present original bad ...


In [23]:
test_clean["Review"] = test_clean['Review'].apply(lambda text: remove_stopwords(text))
test_clean

,Review
0,used favorite game still grinningfacewithsweat
1,every single time put online multiplayer mode ...
2,determined make us watch ads wouldnt issue cho...
3,super great running game
4,hello netease really loved game deserve downlo...
...,...
6995,like playing real football
6996,please fix aim accuracy
6997,fix game ranking chat game sucks communication
6998,give coins


In [20]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

data_clean["text_stemmed"] = data_clean['Review'].apply(lambda text: stem_words(text))
data_clean

,Review,text_stemmed
0,great game kid grew watching pokémon buying po...,great game kid grew watch pokémon buy pokemon ...
1,nice game,nice game
2,balo game bholo laguchi khelibaku gote khusi a...,balo game bholo laguchi khelibaku gote khusi a...
3,worst computer cheating game ive ever played u...,worst comput cheat game ive ever play unfortun...
4,great,great
...,...,...
20995,افضل لعبة و لكن اريد ان يكون شجرة الجواهر,افضل لعبة و لكن اريد ان يكون شجرة الجواهر
20996,nice enjoy,nice enjoy
20997,lots different games depending style also cust...,lot differ game depend style also custom chara...
20998,several frustrating bugs present original bad ...,sever frustrat bug present origin bad port sti...


In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/yugal/nltk_data...


True

In [25]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/yugal/nltk_data...


True

In [26]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

data_clean["text_lemmatized"] = data_clean["Review"].apply(lambda text: lemmatize_words(text))
data_clean

,Review,text_stemmed,text_lemmatized
0,great game kid grew watching pokémon buying po...,great game kid grew watch pokémon buy pokemon ...,great game kid grew watching pokémon buying po...
1,nice game,nice game,nice game
2,balo game bholo laguchi khelibaku gote khusi a...,balo game bholo laguchi khelibaku gote khusi a...,balo game bholo laguchi khelibaku gote khusi a...
3,worst computer cheating game ive ever played u...,worst comput cheat game ive ever play unfortun...,worst computer cheating game ive ever played u...
4,great,great,great
...,...,...,...
20995,افضل لعبة و لكن اريد ان يكون شجرة الجواهر,افضل لعبة و لكن اريد ان يكون شجرة الجواهر,افضل لعبة و لكن اريد ان يكون شجرة الجواهر
20996,nice enjoy,nice enjoy,nice enjoy
20997,lots different games depending style also cust...,lot differ game depend style also custom chara...,lot different game depending style also custom...
20998,several frustrating bugs present original bad ...,sever frustrat bug present origin bad port sti...,several frustrating bug present original bad p...


In [27]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

data_clean["text_lemmatized"] = data_clean["Review"].apply(lambda text: lemmatize_words(text))
data_clean

,Review,text_stemmed,text_lemmatized
0,great game kid grew watching pokémon buying po...,great game kid grew watch pokémon buy pokemon ...,great game kid grow watch pokémon buying pokem...
1,nice game,nice game,nice game
2,balo game bholo laguchi khelibaku gote khusi a...,balo game bholo laguchi khelibaku gote khusi a...,balo game bholo laguchi khelibaku gote khusi a...
3,worst computer cheating game ive ever played u...,worst comput cheat game ive ever play unfortun...,worst computer cheat game ive ever play unfort...
4,great,great,great
...,...,...,...
20995,افضل لعبة و لكن اريد ان يكون شجرة الجواهر,افضل لعبة و لكن اريد ان يكون شجرة الجواهر,افضل لعبة و لكن اريد ان يكون شجرة الجواهر
20996,nice enjoy,nice enjoy,nice enjoy
20997,lots different games depending style also cust...,lot differ game depend style also custom chara...,lot different game depend style also customize...
20998,several frustrating bugs present original bad ...,sever frustrat bug present origin bad port sti...,several frustrate bug present original bad por...


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [245]:
X_train, X_test, y_train, y_test = train_test_split(data_clean['Review'], df['Rating'], test_size=0.2, random_state=30)

In [243]:
from sklearn.metrics import accuracy_score

tfidf = TfidfVectorizer(stop_words='english', max_df=0.8) # stop words removal and max document frequency set to 0.7
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

# Training a Support Vector Machine classifier on the training data
clf = SVC(kernel='poly')
clf.fit(X_train, y_train)

# Predicting on the testing data
y_pred = clf.predict(X_test)

# Evaluating the model's accuracy
accuracy = accuracy_score(y_test, y_pred)

In [244]:
accuracy

0.9304761904761905

In [246]:
tfidf = TfidfVectorizer(stop_words='english', max_df=0.7) # stop words removal and max document frequency set to 0.7
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

In [248]:
from sklearn.ensemble import RandomForestClassifier

clf2 = RandomForestClassifier(max_depth=50,n_estimators=1000, random_state=30,criterion='entropy')
clf2.fit(X_train, y_train)

# Predicting on the testing data
y_pred = clf2.predict(X_test)

# Evaluating the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7652380952380953

In [250]:
pip install pycaret

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
  Using cached imbalanced_learn-0.7.0-py3-none-any.whl (167 kB)
  Using cached scikit_learn-0.23.2-cp38-cp38-manylinux1_x86_64.whl (6.8 MB)
  Using cached numpy-1.20.3-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.6/282.6 kB 783.0 kB/s eta 0:00:000:00:01m eta 0:00:01
  Using cached yellowbrick-1.4-py3-none-any.whl (274 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.3/271.3 kB 1.0 MB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [251]:
from pycaret.classification import *

ImportError: cannot import name '_joblib_parallel_args' from 'sklearn.utils.fixes' (/home/yugal/.local/lib/python3.8/site-packages/sklearn/utils/fixes.py)

In [25]:
s = setup(data_clean, target = 'Review')

Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
Review,Label


ValueError: y contains previously unseen labels: 'worst update march uninstalling'

In [134]:
y_pred1 = clf1.predict(X_test)

In [135]:
accuracy = accuracy_score(y_test, y_pred1)
accuracy

0.7431746031746032

In [77]:
test_clean["text_lemmatized"] = test_clean["Review"].apply(lambda text: lemmatize_words(text))
test_clean

,Review,text_stemmed,text_lemmatized
0,used favorite game still grinningfacewithsweat,use favorit game still grinningfacewithsweat,use favorite game still grinningfacewithsweat
1,every single time put online multiplayer mode ...,everi singl time put onlin multiplay mode load...,every single time put online multiplayer mode ...
2,determined make us watch ads wouldnt issue cho...,determin make us watch ad wouldnt issu choos g...,determine make u watch ad wouldnt issue choose...
3,super great running game,super great run game,super great running game
4,hello netease really loved game deserve downlo...,hello neteas realli love game deserv download ...,hello netease really loved game deserve downlo...
...,...,...,...
6995,like playing real football,like play real footbal,like play real football
6996,please fix aim accuracy,pleas fix aim accuraci,please fix aim accuracy
6997,fix game ranking chat game sucks communication,fix game rank chat game suck commun,fix game rank chat game suck communication
6998,give coins,give coin,give coin


In [223]:
testx = tfidf.transform(test_clean['Review'])

In [224]:
testx

<7000x9185 sparse matrix of type '<class 'numpy.float64'>'
	with 45764 stored elements in Compressed Sparse Row format>

In [225]:
ans = clf.predict(testx)

In [226]:
ans

array([5, 1, 2, ..., 5, 5, 5])

In [227]:
sub1 = pd.DataFrame(ans)
sub1.set_axis(["Rating"],axis=1,inplace=True)


/tmp/ipykernel_77225/1002234618.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  sub1.set_axis(["Rating"],axis=1,inplace=True)


In [228]:
sub1

,Rating
0,5
1,1
2,2
3,5
4,3
...,...
6995,5
6996,3
6997,5
6998,5


In [229]:
sub1.to_csv('file6.csv',index=True)

In [51]:
sample = pd.read_csv("sample_submission.csv")
sample

,Id,Rating
0,0,5
1,1,5
2,2,5
3,3,5
4,4,5
...,...,...
6995,6995,5
6996,6996,5
6997,6997,5
6998,6998,5


In [52]:
sample1 = sample['Rating']
sample1

0       5
1       5
2       5
3       5
4       5
       ..
6995    5
6996    5
6997    5
6998    5
6999    5
Name: Rating, Length: 7000, dtype: int64

In [53]:
accuracy = accuracy_score(sample1, testx)

TypeError: len() of unsized object